In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
import unidecode
import pulp

In [2]:
df_fixtures = pd.read_html("fbref_fixtures_2324.html")[0]
df_fixtures = df_fixtures.dropna(subset='Wk')
df_fixtures = df_fixtures[df_fixtures['xG'] != 'xG']

In [3]:
df_fixtures['xG'] = pd.to_numeric(df_fixtures['xG'])
df_fixtures['xG.1'] = pd.to_numeric(df_fixtures['xG.1'])

In [4]:
filtered_home_xg = df_fixtures[df_fixtures['xG'] < 1]
filtered_away_xg = df_fixtures[df_fixtures['xG.1'] < 1]

count_playing_away = filtered_home_xg['Away'].value_counts()
count_playing_home = filtered_away_xg['Home'].value_counts()

total_counts = count_playing_home.add(count_playing_away, fill_value=0)

In [5]:
df_total_counts = total_counts.reset_index()
df_total_counts.columns = ['Squad', 'xClean']

In [6]:
df = pd.read_html("fbref_standard_2324.html")[0]
df.columns = df.columns.droplevel(0)
df.to_csv("fbref_standard_2324_raw.csv", index=False)

In [7]:
df = pd.read_csv("fbref_standard_2324_filtered.csv")

In [8]:
df = df.merge(df_total_counts, how='left', on='Squad')
df['xClean'] = df['xClean'].fillna(0)

In [9]:
df = df[df['Rk'] != 'Rk']
df['npxG'] = pd.to_numeric(df['npxG'])
df['xAG'] = pd.to_numeric(df['xAG'])
df['90s'] = pd.to_numeric(df['90s'])
df = df[df['Squad'] != 'Burnley']
df = df[df['Squad'] != 'Luton Town']
df = df[df['Squad'] != 'Sheffield Utd']

In [10]:
df_fpl_data = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/cleaned_players.csv")

In [11]:
df1 = df[["Player", "npxG"]]

df2 = df_fpl_data[['first_name', 'second_name', 'element_type', 'now_cost']]

# Function to preprocess names (remove accents, lowercase)
def preprocess_name(name):
    return unidecode.unidecode(name).lower()

df1['processed_name'] = df1['Player'].apply(preprocess_name)
df2['processed_name'] = (df2['first_name'] + ' ' + df2['second_name']).apply(preprocess_name)

# Fuzzy matching
def match_names_with_score(row, choices, scorer=fuzz.token_sort_ratio, threshold=60):
    match, score, _ = process.extractOne(row['processed_name'], choices, scorer=scorer)
    if score >= threshold:
        return match, score
    else:
        return None, None

# Perform the fuzzy matching
choices = df2['processed_name'].tolist()
df1['matched_name'], df1['score'] = zip(*df1.apply(match_names_with_score, axis=1, choices=choices))

# Merge DataFrames
merged_df = pd.merge(df1, df2, left_on='matched_name', right_on='processed_name', how='left')

# Drop intermediate columns if needed
merged_df = merged_df.drop(columns=['processed_name_x', 'processed_name_y'])

C:\Users\Peter\AppData\Local\Temp\ipykernel_3572\3738523934.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['processed_name'] = df1['Player'].apply(preprocess_name)
C:\Users\Peter\AppData\Local\Temp\ipykernel_3572\3738523934.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['processed_name'] = (df2['first_name'] + ' ' + df2['second_name']).apply(preprocess_name)
C:\Users\Peter\AppData\Local\Temp\ipykernel_3572\3738523934.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [12]:
merged_df[merged_df['npxG'] >= 1].to_clipboard() # do some manual cleaning 'off-screen'

In [13]:
df_cleaned = pd.read_csv("clean_fpl_xg_pos.csv")

In [14]:
df_merged = df[['Player', 'Squad', 'npxG', 'xAG', 'xClean', '90s']].merge(df_cleaned[['Player', 'first_name', 'second_name', 'element_type', 'now_cost']])
df_merged = df_merged.rename(columns={"Player": "Name", "Squad": "Club", "element_type": "Position", "now_cost": "Cost"})

In [15]:
df_merged.loc[df_merged['second_name'] == 'Solanke-Mitchell', 'Club'] = 'Tottenham'

In [16]:
leif_davis = {
    'Name': ['Leif Davis'],
    'Club': ['Ipswich'],
    'npxG': [2],
    'xAG': [9],
    'xClean': [8],
    '90s': [36],
    'first_name': ['Leif'],
    'second_name': ['Davis'],
    'Position': ['DEF'],
    'Cost': [45]
}

df_leif_davis = pd.DataFrame(leif_davis)

df_leif_davis


,Name,Club,npxG,xAG,xClean,90s,first_name,second_name,Position,Cost
0,Leif Davis,Ipswich,2,9,8,36,Leif,Davis,DEF,45


In [17]:
df_merged = pd.concat([df_merged, df_leif_davis])

In [18]:
flat_pen_taker_bonus = (99 + 106 + 81 + 106 + 92) / 20 / 5 # number of penalties per season / 20 (for each team) averaged over last 5 seasons
top_bottom_diff = 1.2 # https://www.perplexity.ai/search/number-of-penalties-from-teams-hMulbzGRRaugUOmQUGiNaQ#1 (top ten teams averaged around 6 penalties, 1 higher)
penalty_xg = 0.79 # (opta)
print(f"top ten pen taker bonus: {(flat_pen_taker_bonus + top_bottom_diff) * penalty_xg}")
print(f"bottom ten pen taker bonus: {(flat_pen_taker_bonus - top_bottom_diff) * penalty_xg}")

df_merged['xG'] = df_merged['npxG']
df_merged.loc[df_merged['second_name'] == 'Haaland', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Saka', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Salah', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Heung-min', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
# df_merged.loc[df_merged['second_name'] == 'Solanke-Mitchell', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Palmer', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Borges Fernandes', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Watkins', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Pedro', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Isak', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Mbeumo', 'xG'] += (flat_pen_taker_bonus + top_bottom_diff) * penalty_xg

df_merged.loc[df_merged['second_name'] == 'Unal', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Mateta', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Calvert-Lewin', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Muniz Carvalho', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Gibbs-White', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Paqueta', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg
df_merged.loc[df_merged['second_name'] == 'Sarabia', 'xG'] += (flat_pen_taker_bonus - top_bottom_diff) * penalty_xg

top ten pen taker bonus: 4.7716
bottom ten pen taker bonus: 2.8756


In [19]:
df_merged['points_for_goal'] = df_merged['Position'].apply(lambda x: 6 if x == 'DEF' else 5 if x == 'MID' else 4)
df_merged['points_for_clean'] = df_merged['Position'].apply(lambda x: 4 if x == 'DEF' else 1 if x == 'MID' else 0)

In [20]:
df_merged = df_merged.reset_index(drop=True)

In [21]:
df_merged['xPoints'] = df_merged['xG'] * df_merged['points_for_goal'] + \
    df_merged['xAG'] * 3 + \
    df_merged['xClean'] * df_merged['points_for_clean'] * df_merged['90s'] / 38

In [22]:
max_players = 10  # Total number of players
max_per_club = 3  # Max players per club

# Define possible formations
formations = {
    '3-5-2': {'FWD': 2, 'MID': 5, 'DEF': 3, 'budget': 785},
    # '4-4-2': {'FWD': 2, 'MID': 4, 'DEF': 4, 'budget': 785},
    '3-4-3': {'FWD': 3, 'MID': 4, 'DEF': 3, 'budget': 785},
    # '4-5-1': {'FWD': 1, 'MID': 5, 'DEF': 4, 'budget': 785}
}

best_total_xPoints = 0
best_formation = None
best_selected_players = []
best_captain = None
best_total_cost = 0
best_players_by_position = {}

# Iterate through each formation
for formation_name, max_per_position in formations.items():
    # Create a linear programming problem
    prob = pulp.LpProblem(f"Fantasy_Football_Team_Selection_{formation_name}", pulp.LpMaximize)

    # Decision variables: 1 if the player is selected, 0 otherwise
    x = pulp.LpVariable.dicts("x", df_merged['second_name'], cat="Binary")

    # Decision variables for captains: 1 if the player is selected as captain, 0 otherwise
    c = pulp.LpVariable.dicts("c", df_merged['second_name'], cat="Binary")

    # Objective function: Maximize the total expected goals (xG), considering captaincy
    prob += pulp.lpSum([df_merged.loc[i, 'xPoints'] * (x[df_merged.loc[i, 'second_name']] + c[df_merged.loc[i, 'second_name']]) for i in df_merged.index])

    # Constraint 1: Budget constraint
    prob += pulp.lpSum([df_merged.loc[i, 'Cost'] * x[df_merged.loc[i, 'second_name']] for i in df_merged.index]) <= max_per_position['budget']

    # Constraint 2: Position constraints
    for position, max_count in max_per_position.items():
        if position != 'budget':  # Skip the 'budget' key
            prob += pulp.lpSum([x[df_merged.loc[i, 'second_name']] for i in df_merged.index if df_merged.loc[i, 'Position'] == position]) <= max_count

    # Constraint 3: Total number of players
    prob += pulp.lpSum([x[df_merged.loc[i, 'second_name']] for i in df_merged.index]) == max_players

    # Constraint 4: Club constraints
    clubs = df_merged['Club'].unique()
    for club in clubs:
        prob += pulp.lpSum([x[df_merged.loc[i, 'second_name']] for i in df_merged.index if df_merged.loc[i, 'Club'] == club]) <= max_per_club

    # Constraint 5: Exactly one captain
    prob += pulp.lpSum([c[df_merged.loc[i, 'second_name']] for i in df_merged.index]) == 1

    # Constraint 6: A player can only be captain if they are also in the team
    for i in df_merged.index:
        prob += c[df_merged.loc[i, 'second_name']] <= x[df_merged.loc[i, 'second_name']]

    # Solve the problem
    prob.solve()

    # Calculate total xG and cost for this formation
    total_xPoints = sum(df_merged.loc[i, 'xPoints'] for i in df_merged.index if x[df_merged.loc[i, 'second_name']].varValue == 1) + \
               sum(df_merged.loc[i, 'xPoints'] for i in df_merged.index if c[df_merged.loc[i, 'second_name']].varValue == 1)
    total_cost = sum(df_merged.loc[i, 'Cost'] for i in df_merged.index if x[df_merged.loc[i, 'second_name']].varValue == 1)

    # Check if this formation is better
    if total_xPoints > best_total_xPoints:
        best_total_xPoints = total_xPoints
        best_formation = formation_name
        best_selected_players = [df_merged.loc[i, 'second_name'] for i in df_merged.index if x[df_merged.loc[i, 'second_name']].varValue == 1]
        best_captain = [df_merged.loc[i, 'second_name'] for i in df_merged.index if c[df_merged.loc[i, 'second_name']].varValue == 1][0]
        best_total_cost = total_cost
        
        # Store selected players by position
        best_players_by_position = {
            position: [df_merged.loc[i, 'second_name'] for i in df_merged.index if x[df_merged.loc[i, 'second_name']].varValue == 1 and df_merged.loc[i, 'Position'] == position]
            for position in ['FWD', 'MID', 'DEF']
        }

# Print the best formation and selected players
print(f"Best Formation: {best_formation}")
print("Selected Players:")
for player in best_selected_players:
    print(player)
print(f"\nCaptain: {best_captain}")
print(f"Total Expected Points (xPoints): {best_total_xPoints}")
print(f"Total Cost: {best_total_cost} million")

# Print the selected players by position
print("\nSelected Players by Position:")
for position, players in best_players_by_position.items():
    print(f"{position}s: {', '.join(players)}")

Best Formation: 3-5-2
Selected Players:
Akanji
Díaz
Gibbs-White
Heung-min
Pedro
dos Santos Magalhães
Saka
Salah
Saliba
Watkins

Captain: Salah
Total Expected Points (xPoints): 1199.9301684210527
Total Cost: 785 million

Selected Players by Position:
FWDs: Pedro, Watkins
MIDs: Díaz, Gibbs-White, Heung-min, Saka, Salah
DEFs: Akanji, dos Santos Magalhães, Saliba


In [23]:
df_merged.sort_values('xPoints', ascending=False).head(40)

,Name,Club,npxG,xAG,xClean,90s,first_name,second_name,Position,Cost,xG,points_for_goal,points_for_clean,xPoints
177,Mohamed Salah,Liverpool,15.5,11.8,17.0,28.2,Mohamed,Salah,MID,125,20.2716,5,1,149.373789
176,Bukayo Saka,Arsenal,10.8,10.5,25.0,32.4,Bukayo,Saka,MID,100,15.5716,5,1,130.673789
85,Erling Haaland,Manchester City,22.9,4.3,24.0,28.4,Erling,Haaland,FWD,150,27.6716,4,0,123.586400
158,Cole Palmer,Chelsea,11.1,11.1,9.0,29.0,Cole,Palmer,MID,105,15.8716,5,1,119.526421
90,Son Heung-min,Tottenham,10.4,11.8,9.0,32.6,Son,Heung-min,MID,100,15.1716,5,1,118.979053
124,Gabriel MagalhÃ£es,Arsenal,4.3,0.7,25.0,33.8,Gabriel,dos Santos Magalhães,DEF,60,4.3000,6,4,116.847368
178,William Saliba,Arsenal,1.6,0.2,25.0,38.0,William,Saliba,DEF,60,1.6000,6,4,110.200000
209,Ollie Watkins,Aston Villa,16.8,7.3,8.0,35.7,Ollie,Watkins,FWD,90,21.5716,4,0,108.186400
212,Ben White,Arsenal,1.2,3.8,25.0,33.2,Benjamin,White,DEF,65,1.2000,6,4,105.968421
69,Phil Foden,Manchester City,10.3,8.4,24.0,31.7,Phil,Foden,MID,95,10.3000,5,1,96.721053
